In [10]:

# 라이브러리 불러오기
import requests
import pandas as pd

# 예제 2.7의 get_financials() 함수를 사용 (Dapada apiKey 필요)

my_key = "dpd-e3d49298-fa4dd00070825296fb478bfbb4cf0d57718f3992a30727c9bf58f1eba4c06317"

# 특정 종목(stockCode)의 재무항목(indicatorName) 데이터를 가져오는 함수를 정의
def get_financials(stockCode, indicatorName, apiKey, consolidated=True, ttm=True):
    
    if consolidated:   # 연결 기준
        if ttm:        # Trailing 12 Months (12개월 누적)
            url = f"https://api.dapada.io/company/getConsolidatedFinancialDataByTTM?indicatorName={indicatorName}&stockCode={stockCode}"            
        else:          # 해당 분기 기준 (3개월 집계)
            url = f"https://api.dapada.io/company/getConsolidatedFinancialDataByCUR?indicatorName={indicatorName}&stockCode={stockCode}"       
            
    else:              # 별도 기준
        if ttm:        # Trailing 12 Months (12개월 누적)
            url = f"https://api.dapada.io/company/getSeparatedFinancialDataByTTM?indicatorName={indicatorName}&stockCode={stockCode}"            
        else:          # 해당 분기 기준 (3개월 집계)
            url = f"https://api.dapada.io/company/getSeparatedFinancialDataByCUR?indicatorName={indicatorName}&stockCode={stockCode}"       
           
    headers = {"Authorization": f"{apiKey}"}
    response = requests.get(url, headers=headers)
    result = response.json()
    return pd.DataFrame(result)

# 삼성전자(005930)의 연결 재무제표 (분기 기준)에서 '매출액' 항목의 데이터 수집
df_revenue = get_financials(stockCode='005930',
                            indicatorName='매출액',
                            apiKey=my_key,
                            consolidated=True,
                            ttm=False)


# 열 이름 변경
df_revenue.columns = ['매출액', '분기']

# 분기 열을 인덱스로 설정
df_revenue = df_revenue.set_index('분기', drop=True)

# 2022년 데이터만 선택
df_revenue = df_revenue[df_revenue.index.str.startswith('2023')]

df_revenue

,매출액
분기,
2023-Q3,67404652000000
2023-Q2,60005533000000
2023-Q1,63745371000000


In [13]:
# 삼성전자(005930)의 연결 재무제표(분기 기준)에서 '영업이익' 항목의 데이터 수집
df_margin = get_financials(stockCode='005930',
                           indicatorName='영업이익',
                           apiKey=my_key,
                           consolidated=True,
                           ttm=False)

# 열 이름 변경
df_margin.columns = ['영업이익', '분기']

# 분기 열을 인데그솔 설정
df_margin = df_margin.set_index('분기', drop=True)

# 3분기 데이터만 선택
df_margin = df_margin[df_margin.index.str.endswith('Q3')]

df_margin

,영업이익
분기,
2023-Q3,2433534000000


In [14]:
# 데이터프레임 결합(join)
df_revenue_margin = df_revenue.join(df_margin)
df_revenue_margin

,매출액,영업이익
분기,,
2023-Q3,67404652000000,2.433534e+12
2023-Q2,60005533000000,NaN
2023-Q1,63745371000000,NaN


In [16]:
# 데이터프레임 결합(join) - 교집합
df_revenue_margin2 = df_revenue.join(df_margin, how='inner')
df_revenue_margin2

,매출액,영업이익
분기,,
2023-Q3,67404652000000,2433534000000
